In [1]:
import sys
assert sys.version_info >= (3, 5)

import sklearn
assert sklearn.__version__ >= "0.20"

import numpy as np
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [2]:
import pandas as pd

def load_housing_data(housing_path='data'):
    train_csv_path = os.path.join(housing_path, "train.csv")
    test_csv_path = os.path.join(housing_path, "test.csv")
    return pd.read_csv(train_csv_path), pd.read_csv(test_csv_path)

train, test = load_housing_data()

In [3]:
corr_matrix = train.corr()
corr_matrix["SalePrice"].sort_values(ascending=False)

SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
GarageYrBlt      0.486362
MasVnrArea       0.477493
Fireplaces       0.466929
BsmtFinSF1       0.386420
LotFrontage      0.351799
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
ScreenPorch      0.111447
PoolArea         0.092404
MoSold           0.046432
3SsnPorch        0.044584
BsmtFinSF2      -0.011378
BsmtHalfBath    -0.016844
MiscVal         -0.021190
Id              -0.021917
LowQualFinSF    -0.025606
YrSold          -0.028923
OverallCond     -0.077856
MSSubClass      -0.084284
EnclosedPorch   -0.128578
KitchenAbvGr    -0.135907
Name: SalePr

In [4]:
num_att = list(train.select_dtypes([np.number]).columns)

# att = ['OverallQual', 'GrLivArea']

# to_remove = ['BsmtFinSF2', 'MoSold', '3SsnPorch', 'BsmtHalfBath', 'MiscVal', 'LowQualFinSF', 'YrSold',
#             'PoolArea', 'OverallCond', 'MSSubClass', 'ScreenPorch']

to_remove = ['PoolArea', 'BsmtHalfBath', 'MiscVal', 'LowQualFinSF', 'ScreenPorch', '3SsnPorch',
            'BsmtFinSF2', 'MoSold', 'YrSold',
            'GarageArea', 'TotalBsmtSF']

to_remove = ['PoolArea', 'BsmtHalfBath', 'MiscVal', 'LowQualFinSF', 'ScreenPorch', '3SsnPorch',
            'BsmtFinSF2', 'MoSold']

for t in to_remove:
    num_att.remove(t)
num_att.remove('Id')
num_att.remove('SalePrice')

# num_att = ['OverallQual', 'GrLivArea', 'GarageCars', '1stFlrSF', 'FullBath', 'YearBuilt']

print(num_att)

['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'BsmtFullBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'YrSold']


In [5]:
cat_att = ['BldgType', 'CentralAir', 'Foundation', 
           'PavedDrive', 'SaleCondition']
cat_att = ['Neighborhood', 'BldgType', 'Heating', 'HeatingQC', 'CentralAir', 
           'Street', 'LotShape', 'LandContour', 'LotConfig',
           'LandSlope', 'Condition1', 'HouseStyle', 'RoofStyle', 
           'ExterQual', 'ExterCond', 'Foundation', 'PavedDrive', 'SaleCondition', 
           'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1']

bsmt_cat_att = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1']

# TO ADD = Exterior1st, KitchenQual, Functional, SaleType, Alley, Condition2, BsmtQual

# cat_att = ['Neighborhood', 'BldgType']
print(cat_att)

['Neighborhood', 'BldgType', 'Heating', 'HeatingQC', 'CentralAir', 'Street', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Condition1', 'HouseStyle', 'RoofStyle', 'ExterQual', 'ExterCond', 'Foundation', 'PavedDrive', 'SaleCondition', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1']


In [6]:
for a in num_att:
    median = train[a].median()
    train[a].fillna(median, inplace=True)
    
#     median = test[a].median()
    test[a].fillna(median, inplace=True)

In [7]:
for a in bsmt_cat_att:
    train[a].fillna('NB', inplace=True)
    test[a].fillna('NB', inplace=True)

In [8]:
import seaborn as sns
from scipy.stats import norm
from scipy import stats

train['SalePrice'] = np.log(train['SalePrice'])
train_y = train["SalePrice"].copy()

train['GrLivArea'] = np.log(train['GrLivArea'])
test['GrLivArea'] = np.log(test['GrLivArea'])

train['HasBsmt'] = pd.Series(len(train['TotalBsmtSF']), index=train.index)
train['HasBsmt'] = 0 
train.loc[train['TotalBsmtSF']>0,'HasBsmt'] = 1
train.loc[train['HasBsmt']==1,'TotalBsmtSF'] = np.log(train['TotalBsmtSF'])
test['HasBsmt'] = pd.Series(len(test['TotalBsmtSF']), index=test.index)
test['HasBsmt'] = 0 
test.loc[test['TotalBsmtSF']>0,'HasBsmt'] = 1
test.loc[test['HasBsmt']==1,'TotalBsmtSF'] = np.log(test['TotalBsmtSF'])

train['1stFlrSF'] = np.log(train['1stFlrSF'])
test['1stFlrSF'] = np.log(test['1stFlrSF'])

train['HasBsmtFin'] = pd.Series(len(train['BsmtFinSF1']), index=train.index)
train['HasBsmtFin'] = 0 
train.loc[train['BsmtFinSF1']>0,'HasBsmtFin'] = 1
train.loc[train['HasBsmtFin']==1,'BsmtFinSF1'] = np.log(train['BsmtFinSF1'])
test['HasBsmtFin'] = pd.Series(len(test['BsmtFinSF1']), index=test.index)
test['HasBsmtFin'] = 0 
test.loc[test['BsmtFinSF1']>0,'HasBsmtFin'] = 1
test.loc[test['HasBsmtFin']==1,'BsmtFinSF1'] = np.log(test['BsmtFinSF1'])

train['HasWoodDeck'] = pd.Series(len(train['WoodDeckSF']), index=train.index)
train['HasWoodDeck'] = 0 
train.loc[train['WoodDeckSF']>0,'HasWoodDeck'] = 1
train.loc[train['HasWoodDeck']==1,'WoodDeckSF'] = np.log(train['WoodDeckSF'])
test['HasWoodDeck'] = pd.Series(len(test['WoodDeckSF']), index=test.index)
test['HasWoodDeck'] = 0 
test.loc[test['WoodDeckSF']>0,'HasWoodDeck'] = 1
test.loc[test['HasWoodDeck']==1,'WoodDeckSF'] = np.log(test['WoodDeckSF'])


/home/marek/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [9]:
corr_matrix = train.corr()
corr_matrix["SalePrice"].sort_values(ascending=False)

SalePrice        1.000000
OverallQual      0.817184
GrLivArea        0.730255
GarageCars       0.680625
GarageArea       0.650888
1stFlrSF         0.608947
FullBath         0.594771
YearBuilt        0.586570
YearRemodAdd     0.565608
TotRmsAbvGrd     0.534422
GarageYrBlt      0.495794
Fireplaces       0.489449
MasVnrArea       0.426775
TotalBsmtSF      0.373009
WoodDeckSF       0.343269
LotFrontage      0.335292
OpenPorchSF      0.321053
HasWoodDeck      0.320349
2ndFlrSF         0.319300
HalfBath         0.313982
LotArea          0.257320
BsmtFullBath     0.236224
BsmtUnfSF        0.221985
BedroomAbvGr     0.209044
BsmtFinSF1       0.207962
HasBsmt          0.199634
HasBsmtFin       0.162231
ScreenPorch      0.121208
PoolArea         0.069798
MoSold           0.057329
3SsnPorch        0.054900
BsmtFinSF2       0.004832
BsmtHalfBath    -0.005149
Id              -0.017942
MiscVal         -0.020021
OverallCond     -0.036868
YrSold          -0.037263
LowQualFinSF    -0.037963
MSSubClass  

In [10]:
train2 = train[num_att + cat_att]
train2.info()
test2 = test[num_att + cat_att]
test2.info()

merged_df = pd.concat([train2, test2])
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 50 columns):
MSSubClass       1460 non-null int64
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
MasVnrArea       1460 non-null float64
BsmtFinSF1       1460 non-null float64
BsmtUnfSF        1460 non-null int64
TotalBsmtSF      1460 non-null float64
1stFlrSF         1460 non-null float64
2ndFlrSF         1460 non-null int64
GrLivArea        1460 non-null float64
BsmtFullBath     1460 non-null int64
FullBath         1460 non-null int64
HalfBath         1460 non-null int64
BedroomAbvGr     1460 non-null int64
KitchenAbvGr     1460 non-null int64
TotRmsAbvGrd     1460 non-null int64
Fireplaces       1460 non-null int64
GarageYrBlt      1460 non-null float64
GarageCars       1460 non-null int64
GarageArea       1460 non-nu

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])
train_prepared = num_pipeline.fit_transform(train[num_att])
test_prepared  = num_pipeline.transform(test[num_att])


full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_att),
        ("cat", OneHotEncoder(), cat_att),
    ])

train_test_full_prepared = full_pipeline.fit_transform(merged_df)
train_full_prepared = full_pipeline.transform(train2)
test_full_prepared = full_pipeline.transform(test2)


In [12]:
print(train_prepared.shape)
print(test_prepared.shape)
print(train_full_prepared.shape)
print(test_full_prepared.shape)

(1460, 28)
(1459, 28)
(1460, 158)
(1459, 158)


In [13]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(bootstrap=False, n_estimators=400, max_features=32, random_state=42)
forest_reg.fit(train_full_prepared, train_y)

housing_predictions = forest_reg.predict(train_full_prepared)

forest_mse = mean_squared_error(train_y, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse
# 0.0010380156250445155

0.0010380156250445155

In [14]:
print(housing_predictions[0])
print(train_y[0])

12.247694320220951
12.247694320220994


In [15]:
train_y_normal = np.e**train_y
pred_normal = np.e**housing_predictions

forest_mse = mean_squared_error(train_y_normal, pred_normal)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

138.82069059535877

In [16]:
print(train_y_normal[0])
print(pred_normal[0])

208500.0
208499.9999999911


In [17]:
# forest_reg.feature_importances_

cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = []
for a in cat_encoder.categories_:
    for b in a:
        cat_one_hot_attribs.append(b)
# print(cat_one_hot_attribs)
attributes = num_att + cat_one_hot_attribs
sorted(zip(forest_reg.feature_importances_, attributes), reverse=True)

[(0.1774655044620562, 'OverallQual'),
 (0.12051140499196522, 'GrLivArea'),
 (0.08592509385565401, 'YearBuilt'),
 (0.052684728155289164, 'TotalBsmtSF'),
 (0.052526777750111255, 'GarageArea'),
 (0.05157554130895665, 'GarageCars'),
 (0.05114940274970758, 'TA'),
 (0.04108403564205458, '1stFlrSF'),
 (0.04003627090166764, 'FullBath'),
 (0.027951796494547067, 'Fireplaces'),
 (0.026972763425133776, 'GarageYrBlt'),
 (0.02272791940376236, 'YearRemodAdd'),
 (0.022037498734749177, 'LotArea'),
 (0.01892140117104771, 'BsmtFinSF1'),
 (0.017833377815837506, '2ndFlrSF'),
 (0.01110664041675065, 'Ex'),
 (0.010969986633856631, 'LotFrontage'),
 (0.01068750459783648, 'TotRmsAbvGrd'),
 (0.010305965138501258, 'OverallCond'),
 (0.0096632568729373, 'N'),
 (0.009528512927849387, 'Y'),
 (0.007493372808909645, 'PConc'),
 (0.006878309290910591, 'BsmtUnfSF'),
 (0.0057992055305069075, 'MasVnrArea'),
 (0.005659948835499344, 'OpenPorchSF'),
 (0.005261140790348112, 'MSSubClass'),
 (0.005026090251248021, 'Gd'),
 (0.00498

In [19]:
from sklearn.model_selection import cross_val_score
def display_scores(scores):
    print("scores:", scores)
    print("mean:", scores.mean())
    print("std:", scores.std())

forest_reg = RandomForestRegressor(bootstrap=False, n_estimators=200, max_features=32, random_state=42)
forest_reg.fit(train_full_prepared, train_y)

forest_scores = cross_val_score(forest_reg, train_full_prepared, train_y,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

print(pd.Series(forest_rmse_scores).describe())

scores: [0.13160333 0.12329128 0.13362631 0.16813374 0.1606399  0.11284673
 0.13064878 0.11615301 0.12470106 0.13583349]
mean: 0.13374776292614637
std: 0.01691673644121433
count    10.000000
mean      0.133748
std       0.017832
min       0.112847
25%       0.123644
50%       0.131126
75%       0.135282
max       0.168134
dtype: float64


In [42]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 300, stop = 1300, num = 11)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 200, num = 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True] #, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

{'n_estimators': [300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 31, 52, 73, 94, 115, 136, 157, 178, 200, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True]}


In [43]:
from sklearn.model_selection import RandomizedSearchCV
rf = RandomForestRegressor()

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, 
                               n_iter = 100, cv = 10, verbose=2, random_state=42, 
                               scoring='neg_mean_squared_error',
                               n_jobs = -1)# Fit the random search model
rf_random.fit(train_full_prepared, train_y)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   29.3s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  4.8min
/home/marek/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  9.5min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed: 15.4min finished


RandomizedSearchCV(cv=10, error_score='raise-deprecating',
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators='warn',
                                                   n_jobs=None, oob_score=False,
                                                   random_st...


In [44]:
rf_random.best_params_

{'n_estimators': 1100,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 115,
 'bootstrap': True}

In [45]:
rf_random.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=115,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=1100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [46]:
cvres = rf_random.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)
    

0.14431538823077547 {'n_estimators': 1000, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'auto', 'max_depth': 31, 'bootstrap': True}
0.14427445491293817 {'n_estimators': 900, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'auto', 'max_depth': 52, 'bootstrap': True}
0.1421073040320402 {'n_estimators': 1200, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 136, 'bootstrap': True}
0.1443713816955402 {'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'auto', 'max_depth': 136, 'bootstrap': True}
0.1419607829657424 {'n_estimators': 400, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 52, 'bootstrap': True}
0.14172787685890362 {'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 52, 'bootstrap': True}
0.14159957164965994 {'n_estimators': 1200, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features':

In [47]:
feature_importances = rf_random.best_estimator_.feature_importances_
# feature_importances
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = []
for a in cat_encoder.categories_:
    for b in a:
        cat_one_hot_attribs.append(b)
# print(cat_one_hot_attribs)
attributes = num_att + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

[(0.5493333620664507, 'OverallQual'),
 (0.11210632667081886, 'GrLivArea'),
 (0.04810003055793326, 'TotalBsmtSF'),
 (0.043961219875775226, 'GarageCars'),
 (0.025579078533674145, 'GarageArea'),
 (0.025372359050105726, '1stFlrSF'),
 (0.02257409910964953, 'BsmtFinSF1'),
 (0.01917056462745866, 'YearBuilt'),
 (0.014470491142554021, 'LotArea'),
 (0.012639100283123796, 'OverallCond'),
 (0.009714688891345236, 'YearRemodAdd'),
 (0.0074639268439356705, 'LotFrontage'),
 (0.007410429357136465, 'N'),
 (0.007295686350332567, '2ndFlrSF'),
 (0.007135039409224294, 'Y'),
 (0.005934308598775506, 'Fireplaces'),
 (0.005817125904770048, 'BsmtUnfSF'),
 (0.004767170819496187, 'OpenPorchSF'),
 (0.004368468158165756, 'GarageYrBlt'),
 (0.003405783047026086, 'WoodDeckSF'),
 (0.0031865637632046556, 'MasVnrArea'),
 (0.003037694848921746, 'TotRmsAbvGrd'),
 (0.0028802773870660434, 'MSSubClass'),
 (0.002412219809592119, 'EnclosedPorch'),
 (0.002362485931927873, 'FullBath'),
 (0.0020827648051249623, 'Gd'),
 (0.001971188

In [18]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # sprawdza 12 (3×4) kombinacji hiperparametrów
    {'n_estimators': [100, 300, 350, 400, 450], 'max_features': [16, 32, 64]},
    # następnie sprawdza 6 (2×3) kombinacji z wyłączonym parametrem bootstrap (False)
    {'bootstrap': [False], 'n_estimators': [100, 300, 350, 400, 450], 
     'max_features': [16, 32, 64]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
# przeprowadza proces uczenia na pięciu podzbiorach, czyli łącznie (12+6)*5=90 przebiegów 
grid_search = GridSearchCV(forest_reg, param_grid, cv=10,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(train_full_prepared, train_y)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid=[{'max_features': [16, 32, 64],
 

In [19]:
grid_search.best_params_

{'bootstrap': False, 'max_features': 32, 'n_estimators': 400}

In [20]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=None,
                      max_features=32, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=400,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [21]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)
    
# 13457064367651428

0.1395814188571791 {'max_features': 16, 'n_estimators': 100}
0.13900916407588781 {'max_features': 16, 'n_estimators': 300}
0.13905810606153268 {'max_features': 16, 'n_estimators': 350}
0.1389672784646326 {'max_features': 16, 'n_estimators': 400}
0.13902544546321216 {'max_features': 16, 'n_estimators': 450}
0.13829790976668208 {'max_features': 32, 'n_estimators': 100}
0.13712207733395873 {'max_features': 32, 'n_estimators': 300}
0.13707816570275194 {'max_features': 32, 'n_estimators': 350}
0.13711618187516464 {'max_features': 32, 'n_estimators': 400}
0.13690879206157588 {'max_features': 32, 'n_estimators': 450}
0.13842262366328426 {'max_features': 64, 'n_estimators': 100}
0.13732541621027702 {'max_features': 64, 'n_estimators': 300}
0.137215917509597 {'max_features': 64, 'n_estimators': 350}
0.1374135403323582 {'max_features': 64, 'n_estimators': 400}
0.1372127605438619 {'max_features': 64, 'n_estimators': 450}
0.13656306814767658 {'bootstrap': False, 'max_features': 16, 'n_estimators':

In [22]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

array([5.28541337e-03, 1.07752016e-02, 2.17981034e-02, 1.82957564e-01,
       1.08060096e-02, 8.13988482e-02, 1.90412153e-02, 4.69004141e-03,
       2.01428069e-02, 7.52069677e-03, 4.97468367e-02, 4.04580311e-02,
       1.72528793e-02, 1.47894161e-01, 2.46247479e-03, 4.04380909e-02,
       3.77027751e-03, 4.19194063e-03, 2.31984367e-03, 9.59322109e-03,
       1.94677500e-02, 2.85975357e-02, 5.37651019e-02, 4.35526076e-02,
       4.10749200e-03, 5.46850055e-03, 2.15282683e-03, 2.39884680e-03,
       4.43386735e-05, 4.94320642e-06, 1.18671281e-04, 3.83183822e-04,
       5.62500883e-04, 3.09697710e-04, 1.59969126e-03, 1.52322415e-03,
       1.97652521e-04, 1.06340509e-03, 2.86982762e-04, 1.92032123e-04,
       4.76380379e-04, 3.51599383e-05, 2.63658121e-04, 5.07491650e-04,
       3.54405790e-04, 1.38324369e-03, 1.66979834e-04, 4.15681647e-04,
       1.67369069e-04, 3.38139183e-04, 2.54055574e-04, 1.25329024e-04,
       7.34729631e-05, 1.72645344e-03, 1.27627721e-04, 2.00484874e-04,
      

In [23]:
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = []
for a in cat_encoder.categories_:
    for b in a:
        cat_one_hot_attribs.append(b)
# print(cat_one_hot_attribs)
attributes = num_att + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

[(0.18295756422673984, 'OverallQual'),
 (0.1478941610839692, 'GrLivArea'),
 (0.08139884815910071, 'YearBuilt'),
 (0.05376510194732671, 'GarageCars'),
 (0.05300492755920789, 'TA'),
 (0.04974683668499844, 'TotalBsmtSF'),
 (0.04355260757715006, 'GarageArea'),
 (0.04045803112562272, '1stFlrSF'),
 (0.040438090923223334, 'FullBath'),
 (0.02859753571363083, 'GarageYrBlt'),
 (0.02179810337294522, 'LotArea'),
 (0.02014280689967431, 'BsmtFinSF1'),
 (0.019467749998037766, 'Fireplaces'),
 (0.019041215256983317, 'YearRemodAdd'),
 (0.01725287933590999, '2ndFlrSF'),
 (0.011858556094637632, 'Ex'),
 (0.010806009602525665, 'OverallCond'),
 (0.010775201634408572, 'LotFrontage'),
 (0.010035374402030476, 'N'),
 (0.0095932210883253, 'TotRmsAbvGrd'),
 (0.00952245315850325, 'Y'),
 (0.007612703038973948, 'PConc'),
 (0.007520696766590156, 'BsmtUnfSF'),
 (0.006337138429571781, 'Gd'),
 (0.005468500546976407, 'OpenPorchSF'),
 (0.005285413369032282, 'MSSubClass'),
 (0.0046900414099807376, 'MasVnrArea'),
 (0.0041919

In [48]:
# y_pred = forest_reg.predict(test_full_prepared)
# y_pred = grid_search.predict(test_full_prepared)
y_pred = rf_random.predict(test_full_prepared)
y_pred_normal = np.e**y_pred
sub = pd.DataFrame({'Id': test['Id'], 'SalePrice': y_pred_normal})

In [49]:
sub.to_csv("data/submission_script26_random_rf_2.csv", index=False)

In [38]:
sub.shape

(1459, 2)